# Draft analysis 

---

Group name: Lena Breitberg, Ji Huixiao, Paraskevas Papadopoulos

---


## Introduction

*This section includes an introduction to the project motivation, data, and research question. Include a data dictionary* 

## Setup

In [3]:
import pandas as pd
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Data

## Import data

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/hdm-statistik-lpj/project/main/data/external/immobilienNuernberg.csv')

### Data structure

In [5]:
df

,General.EstateTypeKey,General.ReferenceNumber,General.DistributionTypeKey,General.ConstructionYear,General.LivingSpace,Tealium.object_locationid,Tealium.object_address_is_visible,Tealium.object_zip,Tealium.object_currency,Tealium.object_features,...,USAGE,SUITABILITY,CONDITION,HOUSECONDITION,PARKINGSLOT,ENERGY,BUILDINGTYPE,HEATING,ELEVATOR,EQUIPMENTS
0,WOHNUNG,001/G1/E114,ZUM_KAUF,2023.0,247.07,493480,True,90482,EUR,"['Neubau', 'WANNE', 'gaestewc', 'Bad/WC getren...",...,NaN,NaN,Erstbezug,Neubau,Tiefgarage,NaN,NaN,Fußbodenheizung,Personenaufzug,NaN
1,WOHNUNG,NaN,ZUM_KAUF,1900.0,69.00,493430,False,90461,EUR,"['Dach ausgebaut', 'WANNE', 'gaestewc', 'Bad/W...",...,vermietet,WG-geeignet,renoviert / saniert,NaN,NaN,Gas,NaN,Etagenheizung,NaN,NaN
2,WOHNUNG,NaN,ZUM_KAUF,1972.0,69.46,493465,False,90473,EUR,"['WANNE', 'GEPFLEGT', 'FERN', 'LAMINAT', 'FLIE...",...,vermietet,NaN,gepflegt,NaN,Tiefgarage,Fernwärme,NaN,NaN,NaN,NaN
3,WOHNUNG,X4_301,ZUM_KAUF,2023.0,82.81,493524,True,90429,EUR,"['Neubau', 'DUSCHE', 'Personenaufzug', 'ERSTBE...",...,NaN,NaN,Erstbezug,Neubau,NaN,Blockheizkraftwerk,KfW 55,Fußbodenheizung,Personenaufzug,NaN
4,WOHNUNG,X4_204,ZUM_KAUF,2023.0,75.52,493524,True,90429,EUR,"['Neubau', 'DUSCHE', 'Personenaufzug', 'Erdges...",...,NaN,NaN,Erstbezug,Neubau,NaN,Blockheizkraftwerk,KfW 55,Fußbodenheizung,Personenaufzug,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,WOHNUNG,001/G1/E119,ZUM_KAUF,2023.0,110.32,493480,True,90482,EUR,"['Neubau', 'WANNE', 'gaestewc', 'DUSCHE', 'Kel...",...,NaN,NaN,Erstbezug,Neubau,Tiefgarage,NaN,NaN,Fußbodenheizung,Personenaufzug,NaN
957,WOHNUNG,001/G1/E72,ZUM_KAUF,2023.0,103.11,493480,True,90482,EUR,"['Neubau', 'barriefrei', 'WANNE', 'gaestewc', ...",...,NaN,barrierefrei,Erstbezug,Neubau,Tiefgarage,NaN,NaN,Fußbodenheizung,Personenaufzug,NaN
958,WOHNUNG,NaN,ZUM_KAUF,1965.0,91.00,493430,True,90441,EUR,"['Personenaufzug', 'Zentralheizung', 'LINOLEUM...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zentralheizung,Personenaufzug,NaN
959,WOHNUNG,64415430,ZUM_KAUF,1958.0,34.00,493479,True,90409,EUR,"['DUSCHE', 'Kelleranteil', 'GAS', 'Zentralheiz...",...,frei,NaN,NaN,NaN,NaN,Gas,NaN,Zentralheizung,NaN,möbliert


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 49 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   General.EstateTypeKey                                  961 non-null    object 
 1   General.ReferenceNumber                                832 non-null    object 
 2   General.DistributionTypeKey                            961 non-null    object 
 3   General.ConstructionYear                               961 non-null    float64
 4   General.LivingSpace                                    961 non-null    float64
 5   Tealium.object_locationid                              961 non-null    int64  
 6   Tealium.object_address_is_visible                      961 non-null    bool   
 7   Tealium.object_zip                                     961 non-null    int64  
 8   Tealium.object_currency                           

### Data corrections

### Variable lists

### Data splitting

## Analysis

### Descriptive statistics

### Exploratory data analysis

### Relationships

## Model

### Select model

### Training and validation

### Fit model

### Evaluation on test set

### Save model



Save your model in the folder `models/`. Use a meaningful name and a timestamp.

## Conclusions